In [187]:
import numpy as np
from functools import reduce
from numpy.polynomial import chebyshev as C

In [236]:
small = np.random.randint(1, 50, 20)
medium = np.random.randint(1, 50, 1000)
large = np.random.randint(1, 50, 100000)

power = 2

## `pow` fns

In [280]:
def default_pow(arr, power):
    prd = arr
    for _ in range(power - 1):
        prd = C.chebmul(tgt, arr)
    return prd

In [258]:
def cheb_pow(arr, power):
    zs = C._cseries_to_zseries(arr)
    prd = zs
    for _ in range(power - 1):
        prd = np.convolve(prd, zs)
    return C._zseries_to_cseries(prd)

In [260]:
def reduce_pow(arr, power):
    return reduce(C.chebmul, [arr]*power)

In [267]:
def _binary_pow(func, x, n):
    """
    compute reduce(func, [x]*n) in O(log N) calls to func
    assumes that `func` is associative, and n >= 1
    """
    result = None
    z = x
    while True:
        n, bit = divmod(n, 2)
        if bit:
            if result is None:
                result = z  # optimization - don't initialize result to the identity
            else:
                result = func(result, z)
        if n == 0:
            return result

        z = func(z, z)

def binary_pow(arr, power):
    return _binary_pow(C.chebmul, arr, power)

### Small Size

In [281]:
%timeit default_pow(small, power)

10000 loops, best of 3: 43.9 µs per loop


In [263]:
%timeit cheb_pow(small, power)

The slowest run took 9.18 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 13.4 µs per loop


In [265]:
%timeit reduce_pow(small, power)

The slowest run took 8.96 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 40.8 µs per loop


In [268]:
%timeit binary_pow(small, power)

The slowest run took 5.63 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 41.8 µs per loop


### Medium Size

In [282]:
%timeit default_pow(medium, power)

10000 loops, best of 3: 122 µs per loop


In [269]:
%timeit cheb_pow(medium, power)

1000 loops, best of 3: 1.94 ms per loop


In [271]:
%timeit reduce_pow(medium, power)

1000 loops, best of 3: 698 µs per loop


In [273]:
%timeit binary_pow(medium, power)

1000 loops, best of 3: 679 µs per loop


### Large Size

In [283]:
%timeit default_pow(large, power)

100 loops, best of 3: 7.66 ms per loop


In [275]:
%timeit cheb_pow(large, power)

1 loop, best of 3: 20.6 s per loop


In [278]:
%timeit reduce_pow(large, power)

1 loop, best of 3: 8.07 s per loop


In [279]:
%timeit binary_pow(large, power)

1 loop, best of 3: 9.06 s per loop
